In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
url_list = [
    'https://movie.douban.com/people/98909222/wish?start=0&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=30&sort=time&rating=all&filter=all&mode=list',
    #'https://movie.douban.com/people/98909222/wish?start=60&sort=time&rating=all&filter=all&mode=list'
]

In [3]:
def get_movie_info(movie_entity):
    movie_url = movie_entity.contents[1].contents[1].attrs['href']

    r_movie = requests.get(movie_url, headers=HEADER)
    soup_movie = BeautifulSoup(r_movie.text, 'html.parser')

    return {
        'date': str(str(movie_entity.contents[3].contents[0]).strip()),
        'name': str(soup_movie.find_all('span', {'property': 'v:itemreviewed'})[0].contents[0]),
        'year': int(soup_movie.find_all('span', {'class': 'year'})[0].contents[0].replace('(', '').replace(')', '')),
        'rating': float(soup_movie.find_all('strong', {'class': 'll rating_num', 'property': 'v:average'})[0].contents[0]),
        'votes': int(soup_movie.find_all('span', {'property': 'v:votes'})[0].contents[0]),
        'url': movie_url
    }

In [4]:
def get_page_info(url):
    r_page = requests.get(url, headers=HEADER)
    soup_page = BeautifulSoup(r_page.text, 'html.parser')  # 'lxml'
    movies = soup_page.find_all('div', {'class': 'item-show'})

    for i in range(0, len(movies)):  # 3
        movies_list.append(get_movie_info(movies[i]))
        print('No. ' + str(i))
    del i

In [5]:
movies_list = []
for i in range(0, len(url_list)):
    get_page_info(url_list[i])
del i

No. 0
No. 1
No. 2
No. 3
No. 4
No. 5
No. 6
No. 7
No. 8
No. 9
No. 10
No. 11
No. 12
No. 13
No. 14
No. 15
No. 16
No. 17
No. 18
No. 19
No. 20
No. 21
No. 22
No. 23
No. 24
No. 25
No. 26
No. 27
No. 28
No. 29
No. 0
No. 1
No. 2
No. 3
No. 4
No. 5
No. 6
No. 7
No. 8
No. 9
No. 10
No. 11
No. 12
No. 13
No. 14
No. 15
No. 16
No. 17
No. 18
No. 19
No. 20
No. 21
No. 22


In [6]:
movies_df = pd.DataFrame(data=movies_list)
movies_df['rating_year'] = movies_df['rating'] * movies_df['year']
movies_df['rating_votes'] = movies_df['rating'] * movies_df['votes']
movies_df['rating_year_votes'] = movies_df['rating'] * movies_df['year'] * movies_df['votes']

In [7]:
pd.options.display.max_rows = None
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

movies_df.sort_values('rating_year_votes', ascending=False).reset_index().style.format({
    'url': make_clickable,
    'rating': "{:.1f}",
    'rating_year': "{:.0f}",
    'rating_votes': "{:.0f}",
    'rating_year_votes': "{:.0f}"
})

,index,date,name,year,rating,votes,url,rating_year,rating_votes,rating_year_votes
0,37,2020-07-03,头号玩家 Ready Player One,2018,8.7,1042661,https://movie.douban.com/subject/4920389/,17557,9071151,18305582113
1,6,2020-08-13,龙猫 となりのトトロ,1988,9.2,944663,https://movie.douban.com/subject/1291560/,18290,8690900,17277508405
2,14,2020-08-03,触不可及 Intouchables,2011,9.2,736010,https://movie.douban.com/subject/6786002/,18501,6771292,13617068212
3,50,2020-04-28,教父 The Godfather,1972,9.3,690279,https://movie.douban.com/subject/1291841/,18340,6419595,12659440748
4,49,2020-05-05,教父2 The Godfather: Part Ⅱ,1974,9.2,383260,https://movie.douban.com/subject/1299131/,18161,3525992,6960308208
5,31,2020-07-18,雪国列车 설국열차,2013,7.5,335838,https://movie.douban.com/subject/3071441/,15098,2518785,5070314205
6,30,2020-07-19,E.T. 外星人 E.T.: The Extra-Terrestrial,1982,8.6,266218,https://movie.douban.com/subject/1294638/,17045,2289475,4537739054
7,47,2020-05-13,教父3 The Godfather: Part III,1990,8.9,255376,https://movie.douban.com/subject/1294240/,17711,2272846,4522964336
8,26,2020-07-24,侏罗纪世界2 Jurassic World: Fallen Kingdom,2018,6.7,316696,https://movie.douban.com/subject/26416062/,13521,2121863,4281919938
9,27,2020-07-22,绿里奇迹 The Green Mile,1999,8.8,208486,https://movie.douban.com/subject/1300374/,17591,1834677,3667518923


# 2020-04-11	帕克 Parker